In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
import time
import pickle
import sys
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
%matplotlib inline
from sklearn.metrics import silhouette_samples, silhouette_score

In [2]:
cd '/Users/snehavenkat/Desktop/BDS/ml-100k'

/Users/snehavenkat/Desktop/BDS/ml-100k


In [3]:
#-- Data Preprocessing -1 --
items = pd.read_csv("u.item",sep='|',names=['Movie.ID','Name','release_date','video_release_date','IMDb_URL','unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy',
              'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi',
              'Thriller','War','Western'],index_col=0) # movies information
items = items.drop(['release_date','video_release_date','IMDb_URL','unknown'],axis=1)
genre = pd.read_csv("u.genre",names=['Genre','ID'],sep='|') # movie genre information

In [4]:
#-- Data Preprocessing -2 ---
rate = pd.read_csv("u.data",sep='\t',names=['userId','movieId','rating','timestamp'],index_col='userId')
rate = rate[rate.movieId != 267]
rate = rate[rate.movieId != 1373]
rate = rate.sort_index()
ratings = pd.DataFrame(index=np.unique(rate.index.values),columns=[x for x in range(1,1683)])
ratings = ratings.fillna(0)
for index,row in rate.iterrows():
    ratings.loc[index,row['movieId']] = row['rating']

In [5]:
test_time = time.time()
kmeans = KMeans(n_clusters=4)
kmeans.fit(ratings,)
# pp = pickle.dumps(kmeans)
# print(sys.getsizeof(pp)) #-- Saving the model
print("--- Train time is %s seconds ---" % (time.time() - test_time)) #Calculating Train time

--- Train time is 1.46183300018 seconds ---


In [6]:
labels = kmeans.labels_#labels
centroids = kmeans.cluster_centers_#cluster centers

In [7]:
# k means determine k
distortions = []
K = range(2,11)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(ratings,)
    kmeanModel.fit(ratings,)
    cluster = kmeanModel.labels_#labels
    sil_avg = silhouette_score(ratings, cluster)
    print("Sil score: ",sil_avg)
    print "&quot;For number of clusters: &quot;",str(k),"&quot; average sil score:&quot;",str(sil_avg)
    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(ratings, cluster)
#     distortions.append(sum(np.min(cdist(ratings, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / ratings.shape[0])

# ---- Plot the elbow ---
# plt.plot(K, distortions, 'bx-')
# plt.xlabel('k')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method showing the optimal k')
# plt.show()

('Sil score: ', 0.23427817069118068)
&quot;For number of clusters: &quot; 2 &quot; average sil score:&quot; 0.234278170691
('Sil score: ', 0.14701197951948272)
&quot;For number of clusters: &quot; 3 &quot; average sil score:&quot; 0.147011979519
('Sil score: ', 0.13141090928109175)
&quot;For number of clusters: &quot; 4 &quot; average sil score:&quot; 0.131410909281
('Sil score: ', 0.020106295083060728)
&quot;For number of clusters: &quot; 5 &quot; average sil score:&quot; 0.0201062950831
('Sil score: ', 0.022359759896208045)
&quot;For number of clusters: &quot; 6 &quot; average sil score:&quot; 0.0223597598962
('Sil score: ', 0.0015693545970040252)
&quot;For number of clusters: &quot; 7 &quot; average sil score:&quot; 0.001569354597
('Sil score: ', 0.022195664464648195)
&quot;For number of clusters: &quot; 8 &quot; average sil score:&quot; 0.0221956644646
('Sil score: ', 0.02101694634924433)
&quot;For number of clusters: &quot; 9 &quot; average sil score:&quot; 0.0210169463492
('Sil s

In [8]:
pd.set_option('display.max_rows', ratings.describe().shape[1])
pd.DataFrame(centroids)

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.461538,0.723077,0.538462,2.538462,0.415385,0.507692,2.753846,2.815385,2.707692,1.569231,...,2.168404e-18,6.153846e-02,2.168404e-18,2.602085e-18,2.168404e-18,1.301043e-18,2.168404e-18,2.602085e-18,2.168404e-18,2.168404e-18
1,3.628866,2.237113,1.103093,2.721649,1.402062,0.051546,3.144330,2.319588,2.072165,0.618557,...,3.903128e-18,6.938894e-18,3.903128e-18,3.469447e-18,3.903128e-18,1.734723e-18,3.903128e-18,3.469447e-18,3.092784e-02,3.092784e-02
2,2.342857,0.587755,0.240816,1.167347,0.293878,0.044898,1.848980,1.514286,1.204082,0.224490,...,1.224490e-02,9.540979e-18,1.224490e-02,8.163265e-03,1.224490e-02,2.385245e-18,-1.301043e-17,4.770490e-18,-1.301043e-17,-1.301043e-17
3,1.242086,0.022346,0.134078,0.050279,0.091248,0.081937,1.027933,0.178771,0.918063,0.230912,...,1.474515e-17,-1.647987e-17,1.474515e-17,-8.239937e-18,1.474515e-17,1.862197e-03,5.586592e-03,3.724395e-03,1.474515e-17,1.474515e-17


In [9]:
#-- Inputs for determinging top rated movies -- 
movie_names = items.Name
genre_types = items.columns[1:]
items_array = np.array(items)#user rating and movie title info
movie_genre = np.array(items[genre_types])

In [10]:
def FiveStarMovies(centroids, labels, k, NTerms):
    five_star_movies = []
    cluster_movies = []
    for i in range(k):#k is no. of clusters
        cluster_members = ratings[labels==i]#get the members of the ith cluster
        print 'cluster',i+1,'size: ', cluster_members.shape[0]#get the number of members in the ith cluster
        movie_total_rating = np.array(cluster_members.sum(0)).astype(float)#total ratings for all movies in the ith cluster
        #print(movie_total_rating)
        movie_rating_count = np.array((cluster_members!=0).sum(0)).astype(float)#total no. of users who rated the movies
        movie_average_rating = np.divide(movie_total_rating, movie_rating_count)#average ratings of all movies
        movie_average_rating[np.isnan(movie_average_rating)] = 0
        rated_movie_list = sorted(zip(movie_names, movie_average_rating), key=lambda x: int(x[1]), reverse=True)
        to_append = []
        cluster_append = []
        for j in rated_movie_list:#for each movie
            if j[1] == 5:#if avg rating is highest
                to_append.append(j[0])#append highest rated movies
            if j[1] != 0:
                cluster_append.append(j[0])#all movie average ratings which are rated
        five_star_movies.append(to_append)#highest rated movies in that cluster
        cluster_movies.append(cluster_append)#all movie average ratings which are rated
    print "\n"
    # most rated movies in each cluster
    k = 0
    for m in five_star_movies:#for each highly rated movie
        prototype = list(np.zeros(len(genre_types)))#placeholders for each genre: fill with 0s
        for n in items_array:#user rating and movie title info    
            for i in m:
                if i == n[0]:#5 star movie title?
                    prototype += n[1:]#increment genre of that movie
        print "Representative Movies for ",k+1,":"
        for i in range(5):
            print m[i]#print highly rated movies
        print "\n"
        prototype = map(int, prototype)
        movie_genre_to_use = sorted(zip(genre_types, prototype), key=lambda x:int(x[1]), reverse=True)#types of genres and placeholder count array
        print "Cluster", k+1, "top genre: "
        for i in movie_genre_to_use[:NTerms]:#print top rated genres in each cluster
            print i[0]
        print "\n"
        k += 1
    return five_star_movies, cluster_movies

In [11]:
#Calculate test time
test_time = time.time()
five_star_movies, cluster_movies = FiveStarMovies(centroids, labels, 4, 5)
print("--- Test time is %s seconds ---" % (time.time() - test_time))

cluster 1 size:  65
cluster 2 size:  97
cluster 3 size:  245
cluster 4 size:  537


Representative Movies for  1 :
Stalingrad (1993)
Baby-Sitters Club, The (1995)
Timecop (1994)
Great Day in Harlem, A (1994)
Two or Three Things I Know About Her (1966)


Cluster 1 top genre: 
Drama
Comedy
Action
Animation
Documentary


Representative Movies for  2 :
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Maya Lin: A Strong Clear Vision (1994)
Deceiver (1997)
Meet John Doe (1941)
Telling Lies in America (1997)


Cluster 2 top genre: 
Drama
Crime
Comedy
Documentary
Romance


Representative Movies for  3 :
Maya Lin: A Strong Clear Vision (1994)
Turbo: A Power Rangers Movie (1997)
Perfect Candidate, A (1996)
Washington Square (1997)
Winter Guest, The (1997)


Cluster 3 top genre: 
Drama
Comedy
Action
Documentary
Sci-Fi


Representative Movies for  4 :
Faster Pussycat! Kill! Kill! (1965)
Aristocats, The (1970)
D3: The Mighty Ducks (1996)
Vanya on 42nd Street (1994)
Down by Law (1986)


Cluster 